In [34]:
! sudo apt install openjdk-8-jdk
! sudo update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
! pip install language-check -qq
! pip install pycontractions -qq

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jre x11-utils
Suggested packages:
  openjdk-8-demo openjdk-8-source visualvm icedtea-8-plugin mesa-utils
The following NEW packages will be installed:
  fonts-dejavu-core fonts-dejavu-extra libatk-wrapper-java
  libatk-wrapper-java-jni libxxf86dga1 openjdk-8-jdk openjdk-8-jre x11-utils
0 upgraded, 8 newly installed, 0 to remove and 25 not upgraded.
Need to get 4,942 kB of archives.
After this operation, 13.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libxxf86dga1 amd64 2:1.1.4-1 [13.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 fonts-dejavu-core all 2.37-1 [1,041 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/main amd64 fonts-dejavu-extra all 2.37-1 [1,953 kB

In [35]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

SystemError: ignored

In [0]:
# Import statements
import pandas as pd
import numpy as np
import pprint as pp
import json
from pandas.io.json import json_normalize
import re
from timeit import default_timer

# Preprocessing
from pycontractions import Contractions

# Tokenization imports
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

# Puncuation & lower case
import string #punctuation removal

# Stop words
from nltk.corpus import stopwords

# Stemming
from nltk.stem.snowball import SnowballStemmer

#import flat_table

# Data extraction

In [0]:
# Load JSON file into dataframe
data = pd.read_json('/content/drive/My Drive/frames.json')
df = pd.DataFrame(data)

In [16]:
df.head() # Looks like turns and labels are nested dictionaries

,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


In [17]:
df['id'].nunique() # There are 1369 unique ID for the dialogue

1369

In [0]:
# labels is a nested dictionary
labels_df = json_normalize(df['labels'])

In [19]:
labels_df.head()

,userSurveyRating,wizardSurveyTaskSuccessful
0,4.0,True
1,3.0,True
2,2.0,False
3,5.0,True
4,5.0,True


`userSurveyRating` - value that represents the user’s satisfaction with the Wizard’s service, ranging from 1 – complete dissatisfaction to 5 – complete satisfaction

`wizardSurveyTaskSuccessful` - boolean which is true if the wizard thinks at the end of the dialogue that the user’s goal was achieved

In [0]:
# Attempted json_normalize but kept getting the error: 'list' object has no attribute 'values'
# This happens because of list enclosing each of the dictionary.
#   - Multiple dictionaries with matching keys enclosed by a list.
#   - A dictionary with keys as columns and values in the form of list.

# Thus we try another approach
text_list = []
for item in df['turns']:
  text_list.append(item)
turns_df = pd.DataFrame(text_list)

In [21]:
turns_df.head(2) # Every row is a dialogue 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47
0,{'text': 'I'd like to book a trip to Atlantis ...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'Yes, how about going to Neverland fr...","{'db': {'result': [[], [], [], [], [], []], 's...",{'text': 'I have no flexibility for dates... b...,"{'db': {'result': [[]], 'search': [{'ORIGIN_CI...",{'text': 'I suppose I'll speak with my husband...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"{'text': 'Hello, I am looking to book a vacati...",{'db': {'result': [[{'trip': {'returning': {'d...,{'text': 'What about a trip from Gotham City t...,{'db': {'result': [[{'trip': {'returning': {'d...,{'text': 'Would any packages to Mos Eisley be ...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'You know what, I'd like to try and v...",{'db': {'result': [[{'trip': {'returning': {'d...,{'text': 'Do you have any trips from Gotham Ci...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'No, that's too far for me. I need a ...",{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'How many days would I be in Kobe?', ...","{'db': {'result': [], 'search': []}, 'text': '...",{'text': 'What would the price be if I shorten...,{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'Ok, then I would like to purchase th...",{'db': {'result': [[{'trip': {'returning': {'d...,"{'text': 'Yes, I would like to book this packa...","{'db': {'result': [], 'search': []}, 'text': '...",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [0]:
turns_df.replace(np.nan,'',inplace=True) # Replace all null values

In [0]:
turns_df_T = turns_df.transpose() # Transposed the dataframe to have proper conversation turns

In [0]:
# Iterates through the data frame and gets all values with a text key
convo_text = ''
for i in turns_df_T:                                 # 1369 columns
  for j in range(0,48):                              # 48 rows
    try:
      get_text = turns_df_T[i][j].get('text')        # Gets the value from the corresponding key
      convo_text += get_text + ' '                   # Concatenates all text
    except AttributeError:
      convo_text += ''                               # Concatenates '' if there is no text key - used for switching conversations

# Data Preprocessing

In [25]:
# Conversation sample
pp.pprint(convo_text[0:888]) 

("I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, "
 '2016 for 8 adults. I have a tight budget of 1700. Hi...I checked a few '
 'options for you, and unfortunately, we do not currently have any trips that '
 'meet this criteria.  Would you like to book an alternate travel option? Yes, '
 'how about going to Neverland from Caprica on August 13, 2016 for 5 adults. '
 'For this trip, my budget would be 1900. I checked the availability for this '
 'date and there were no trips available.  Would you like to select some '
 'alternate dates? I have no flexibility for dates... but I can leave from '
 'Atlantis rather than Caprica. How about that? I checked the availability for '
 'that date and there were no trips available.  Would you like to select some '
 "alternate dates? I suppose I'll speak with my husband to see if we can "
 "choose other dates, and then I'll come back to you.Thanks for your help ")


## Contractions

In [26]:
!wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

--2020-03-24 19:09:23--  https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.20.67
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.20.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1647046227 (1.5G) [application/x-gzip]
Saving to: ‘GoogleNews-vectors-negative300.bin.gz’

GoogleNews-vectors- 100%[===================>]   1.53G  47.5MB/s    in 34s     

2020-03-24 19:09:57 (46.5 MB/s) - ‘GoogleNews-vectors-negative300.bin.gz’ saved [1647046227/1647046227]



In [75]:
len(convo_text[0:134175])/len(convo_text) # Approx. 10% of dataset

0.10124420774473858

In [125]:
pp.pprint(convo_text[134000:134175]) # Logical end of conversation

('Would you like me to book this package? Hold your horse! It sounds like a '
 'great deal but I need to check with my bride first. I will get back to you.  '
 'Thank you for your help.')


In [0]:
convo_text_sample = convo_text[0:134175]
#convo_text_sample = convo_text[0:round(len(convo_text) * 0.10)]

In [0]:
# Expand contractions
# - Load semantic vector model in gensim keyedvectors format from disk
cont = Contractions('GoogleNews-vectors-negative300.bin.gz')

In [79]:
start = default_timer()
expand_convo_text_sample_false = list(cont.expand_texts([convo_text_sample], precise=False)) #this part takes forever
end = default_timer()
print(f"elapsed time: {end-start}s")

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:410: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


elapsed time: 736.7734251849997s


In [96]:
start = default_timer()
expand_convo_text_sample_false = str(list(cont.expand_texts([convo_text_sample], precise=False))) #this part takes forever
end = default_timer()
print(f"elapsed time: {end-start}s")

elapsed time: 609.4058134629995s


In [97]:
print(f"elapsed time: {(end-start)/60} min")

elapsed time: 10.156763557716658 min


## Tokenization

In [84]:
nltk.download('punkt') # Punkt sentence tokenizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [99]:
#split into sentences
tokenized_sent = sent_tokenize(expand_convo_text_sample_false)
pp.pprint(tokenized_sent[0:13])

['["I would like to book a trip to Atlantis from Caprica on Saturday, August '
 '13, 2016 for 8 adults.',
 'I have a tight budget of 1700.',
 'Hi...I checked a few options for you, and unfortunately, we do not currently '
 'have any trips that meet this criteria.',
 'Would you like to book an alternate travel option?',
 'Yes, how about going to Neverland from Caprica on August 13, 2016 for 5 '
 'adults.',
 'For this trip, my budget would be 1900.',
 'I checked the availability for this date and there were no trips available.',
 'Would you like to select some alternate dates?',
 'I have no flexibility for dates... but I can leave from Atlantis rather than '
 'Caprica.',
 'How about that?',
 'I checked the availability for that date and there were no trips available.',
 'Would you like to select some alternate dates?',
 'I suppose I will speak with my husband to see if we can choose other dates, '
 'and then I will come back to you.Thanks for your help Hello, I am looking to '
 'book a v

In [100]:
len(tokenized_sent)

2517

In [101]:
#split into words
tokenized_words=[]
tokenized_words.extend(word for word in word_tokenize(str(tokenized_sent)))
pp.pprint(tokenized_words[0:20])

['[',
 "'",
 '[',
 '``',
 'I',
 'would',
 'like',
 'to',
 'book',
 'a',
 'trip',
 'to',
 'Atlantis',
 'from',
 'Caprica',
 'on',
 'Saturday',
 ',',
 'August',
 '13']


## Punctuation & Lower Case

In [0]:
string.punctuation = string.punctuation + '``' + '...'

In [117]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~````[\'``\', \'...\']``...'

In [118]:
# Removes punctuation
no_punct = [token for token in tokenized_words if token not in string.punctuation] 
print(no_punct[0:20])

['I', 'would', 'like', 'to', 'book', 'a', 'trip', 'to', 'Atlantis', 'from', 'Caprica', 'on', 'Saturday', 'August', '13', '2016', 'for', '8', 'adults', "'I"]


In [119]:
# Lowercase text
data_lower = [word.lower() for word in no_punct]
print(data_lower[0:20])

['i', 'would', 'like', 'to', 'book', 'a', 'trip', 'to', 'atlantis', 'from', 'caprica', 'on', 'saturday', 'august', '13', '2016', 'for', '8', 'adults', "'i"]


## Stop Words

In [120]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [121]:
stop_words = stopwords.words('english')
#add_stop_words = ['\'d]
print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [0]:
filtered_sent = []
for word in data_lower:
  if word not in stop_words:
    filtered_sent.append(word)

In [123]:
filtered_sent[0:20]

['would',
 'like',
 'book',
 'trip',
 'atlantis',
 'caprica',
 'saturday',
 'august',
 '13',
 '2016',
 '8',
 'adults',
 "'i",
 'tight',
 'budget',
 '1700',
 "'hi",
 'checked',
 'options',
 'unfortunately']

## Stemming